In [1]:
import os, tempfile, urllib, zipfile, pandas, absl

import pandas

import tensorflow as tf

from tfx.components import CsvExampleGen

tf.get_logger().setLevel('ERROR')

In [2]:
url = 'data/diabetic_data.csv'

data = pandas.read_table(url, header=0, na_values='?', sep=',')

data.shape

(101766, 50)

In [3]:
data.iloc[:5, :5]

,encounter_id,patient_nbr,race,gender,age
0,2278392,8222157,Caucasian,Female,[0-10)
1,149190,55629189,Caucasian,Female,[10-20)
2,64410,86047875,AfricanAmerican,Female,[20-30)
3,500364,82442376,Caucasian,Male,[30-40)
4,16680,42519267,Caucasian,Male,[40-50)


In [4]:
data.iloc[:5, 45:]

,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,No,No,No,No,NO
1,No,No,Ch,Yes,>30
2,No,No,No,Yes,NO
3,No,No,Ch,Yes,NO
4,No,No,Ch,Yes,NO


In [5]:
null = data.isnull().sum()

null.sum()

192849

In [6]:
from sklearn.model_selection import train_test_split

test_part = 0.3

train, test = train_test_split(data, test_size=test_part, shuffle=False)

test, eval = train_test_split(test, test_size=0.5, shuffle=False)

In [7]:
eval = eval.drop('readmitted', axis='columns')

eval.shape

(15265, 49)

In [8]:
from tensorflow_data_validation import StatsOptions

remopal = ['encounter_id', 'patient_nbr']

stats_cols = [col for col in data.columns if (col not in remopal)]

stats_options = StatsOptions(feature_allowlist=stats_cols)

In [9]:
from tensorflow_data_validation import generate_statistics_from_dataframe

train_stats = generate_statistics_from_dataframe(train, stats_options=stats_options)

In [10]:
len(train_stats.datasets[0].features)

48

In [11]:
train_stats.datasets[0].num_examples

71236

In [12]:
train_stats.datasets[0].features[0].path.step[0]

'race'

In [13]:
from tensorflow_data_validation import visualize_statistics

visualize_statistics(train_stats)

In [14]:
from tensorflow_data_validation import infer_schema, display_schema

schemas = infer_schema(train_stats)

display_schema(schemas)

In [15]:
len(schemas.feature)

48

In [ ]:
test_stats = generate_statistics_from_dataframe(test, stats_options=stats_options)

visualize_statistics(test_stats)

In [17]:
statlist = {}

statlist['lhs_statistics'] = test_stats; statlist['lhs_name'] = 'Test Data'

statlist['rhs_statistics'] = train_stats; statlist['rhs_name'] = 'Train Data'

visualize_statistics(**statlist)

In [18]:
from tensorflow_data_validation import validate_statistics

anomalies = validate_statistics(statistics=train_stats, schema=schemas)

TotalAnomalies = len(anomalies.anomaly_info.items())

TotalAnomalies

0

In [19]:
anomalies = validate_statistics(statistics=test_stats, schema=schemas)

TotalAnomalies = len(anomalies.anomaly_info.items())

TotalAnomalies

2